# Event stream warping

This notebook implements our event stream warping (ESW) algorithm. It is a generalization for dynamic time warping, where we dont assume that events are linearly ordered, but rather partially ordered.

We also implement parsing of TestCases here since their structure is coupled with the ESW implementation.

In [ ]:
#| default_exp stream_warping

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2
# %load_ext rich

## Imports

In [ ]:
#| export
import os
import json
from frozendict import frozendict
from collections import defaultdict

from pydantic import BaseModel, ConfigDict, field_validator, Field
from typing import List, Any, Dict, Callable,Set, Optional

import numpy as np
import itertools as it
import re
import asyncio

from constraint import Problem,FunctionConstraint
from bidict import bidict

import logging 
from stringdale.core import checkLogs,await_all
from stringdale.mappings import access, parse_edge_descriptor


In [ ]:
#| export
logger = logging.getLogger(__name__)

In [ ]:
import pytest

## Mapping labels to fresh variables for CSP

In [ ]:
#| export
def int_to_excel_col(n):
    if n < 0:
        raise ValueError("Number must be non-negative")
    
    result = ""
    n += 1  # Adjust because Excel columns start at 1, not 0
    
    while n > 0:
        n -= 1  # Adjust for 0-based indexing
        result = chr(n % 26 + ord('A')).lower() + result
        n //= 26
        
    return result

In [ ]:

assert int_to_excel_col(0) == "a"
assert int_to_excel_col(25) == "z"
assert int_to_excel_col(26) == "aa"
assert int_to_excel_col(27) == "ab"
assert int_to_excel_col(51) == "az"
assert int_to_excel_col(52) == "ba"
assert int_to_excel_col(701) == "zz"
assert int_to_excel_col(702) == "aaa"


In [ ]:
#| export
class LabelToVar():
    def __init__(self):
        self.label_to_var = bidict()
        self.label_to_index = bidict()

    def add_label(self,label:str,idx:int):
        self.label_to_var[label] = int_to_excel_col(idx)
        self.label_to_index[label] = idx

    def get_label(self,col:str) -> str:
        return self.label_to_var.inverse[col]

    def get_index(self,label:str) -> int:
        return self.label_to_index[label]

    def get_col(self,label:str) -> int:
        return self.label_to_var[label]


In [ ]:
label_to_var = LabelToVar()
label_to_var.add_label("x",0)
label_to_var.add_label("y",1)
label_to_var.add_label("z",2)

assert label_to_var.get_col("x") == "a"
assert label_to_var.get_col("y") == "b"
assert label_to_var.get_col("z") == "c"

assert label_to_var.get_label("a") == "x"
assert label_to_var.get_label("b") == "y"
assert label_to_var.get_label("c") == "z"

assert label_to_var.get_index("x") == 0
assert label_to_var.get_index("y") == 1
assert label_to_var.get_index("z") == 2

## Mock comparison functions

In [ ]:
#| export

async def word_overlap(result: str, expected: str,**kwargs) -> float:
    """
    Calculate the distance between result and expected strings based on word overlap.
    Returns a value between 0 and 1, where:
    - 0 means perfect match (all words from result are in expected)
    - 1 means no overlap (no words from result are in expected)
    
    Args:
        result (str): The string to check words from
        expected (str): The string to check words against
        
    Returns:
        float: Distance metric between 0 and 1
    """
    if not isinstance(result,str) or not isinstance(expected,str):
        return np.inf
    # Convert both strings to lowercase and split into words
    result_words = set(result.lower().split())
    expected_words = set(expected.lower().split())
    
    # If result is empty, return 1.0 (maximum distance)
    if not result_words:
        return 1.0
    
    # Calculate overlap
    overlap = len(result_words.intersection(expected_words))
    total = len(result_words)
    
    # Calculate distance (1 - percentage)
    distance = 1.0 - (overlap / total)
    
    return distance

In [ ]:
# Example 1
result = "The quick brown fox"
expected = "The lazy brown dog"
assert await word_overlap(result, expected) == 0.5  # Output: 0.5 (2 out of 4 words match)

# Example 2
result = "Hello world"
expected = "Hello there world"
assert await word_overlap(result, expected) == 0.0  # Output: 0.0 (all words match)

# Example 3
result = "Python programming"
expected = "Java development"
assert await word_overlap(result, expected) == 1.0  # Output: 1.0 (no words match)

In [ ]:
#| export
def regex(out: str, expected: str,mismatch_penalty=1.0,**kwargs) -> float:
    """
    Compare a string against a regex pattern.
    Returns 0 if the regex matches, 1 if it doesn't.
    
    Args:
        out (str): The string to check
        expected (str): The regex pattern to match against
        
    Returns:
        float: 0 if match, 1 if no match
    """
    if not isinstance(expected,str):
        raise ValueError("expected must be a string")
    if not isinstance(out,str):
        return np.inf
    try:
        if re.search(expected, out,flags=re.IGNORECASE) is not None:
            return 0
        return mismatch_penalty
    except Exception:
        return mismatch_penalty


In [ ]:
# Basic matching
assert regex("hello world", "hello") == 0.0  # Simple substring match
assert regex("hello world", "^hello") == 0.0  # Start anchor
assert regex("hello world", "world$") == 0.0  # End anchor
assert regex("hello world", "hello.*world") == 0.0  # Pattern with wildcard

# Non-matching
assert regex("hello world", "goodbye") == 1.0  # No match
assert regex("hello world", "^world") == 1.0  # Wrong position
assert regex("hello world", "hello$") == 1.0  # Wrong position with anchor

# Pattern errors and edge cases
assert regex("hello world", "(unclosed") == 1.0  # Invalid regex pattern
assert regex("hello world", "") == 0.0  # Empty pattern matches anything
assert regex("", ".*") == 0.0  # Empty string matches wildcard
assert regex("", "") == 0.0  # Empty string matches empty pattern

# Case sensitivity
assert regex("Hello World", "hello") == 0.0  # Case-insensitive by default



## Running example

In [ ]:
# TODO
# in the future we can allow multiple inputs and expected blocks 
# so that people can interweave inputs and expectations

In [ ]:

example_case = """
inputs:
  - content: "hello world"
test_nodes:
  # we give the name of the trace node
  - name: node_a
    # we can also give a label to the node so we can refer to it later
    label: node_a1
    
    # we describe what output we expect from the node using accessors as keys
    # the value is what we expect the accessor to return
    conditions:
      - key: b.c
        value: |
          jimmy went
          to the store

  - name: node_b
    # we can give multiple comparisons to the same node, using different accessors
    conditions:
      - key: d.e
        value: jimmy
        # we can specify the comparison function to use
        func: "regex"
      - key: f.g
        value: "is a good boy"
        func: "chat"
        # we can pass kwargs to the comparison function
        kwargs:
          case_sensitive: false

  # we can also give a regex to match the node name
  - name: node_.*
    conditions:
      - key: .
        value: "store"
    # using the $parallel key we can specify that this node is expected in parallel with the previous node
    # since we do not know which trace will be logged first
    parallel: true
    label: node_z
    
  - name: node_c
    # we can specify more complex ordering constraints on nodes using before and after keys
    # before and after take a list of node labels
    # in this case we say that node_c should be after node_a1 and before node_z
    after: 
      - node_a1
    before: 
      - node_z
    conditions:
      - key: b.c
        value: "store"
    
"""



In [ ]:
#| export
from stringdale.mappings import parse_accessor

In [ ]:
example_trace_log = [
    {
        # should be ignored
        "name": "Start",
        "output": "hello world"
    },
    {
        "name": "node_a",
        "output": {'b':{'c':"jimmy went\nto the store\nto buy some milk"}}
    },
    {
        # first option to node c
        "name": "node_c",
        "output": {'b':{'c':"store is good"}}
    },
    {
        # shouldnt match
        "name": "node_a2",
        "output": {'b':{'d':"store"}}
    },
    {
        # first option to node_z
        "name": "node_x",
        "output": "store"
    },
    {
        "name": "node_b",
        "output": {
            'f':{'g':"is a good boy"},
            'd':{'e':"jimmy"}
            }
    },
    {   
        # second option to node c, only relevant if node_* matches to node_y
        "name": "node_c",
        "output": {'b':{'c':"store is good but not good enough"}}
    },
    {
        # second option to node_z
        "name": "node_y",
        "output": "stores"
    },
]

# expected labels: [noda_a1,1 (b) ,node_z,3 (c)] # c needs to be before z
# a:[1], c:[2,6] , b:[5] z:[4,7]

In [ ]:
possible_mappings = {
    frozendict({'node_a1':1,'1':5,'node_z':4,'3':2}), 
    frozendict({'node_a1':1,'1':5,'node_z':7,'3':2}),
    frozendict({'node_a1':1,'1':5,'node_z':7,'3':6}),
}

best_mapping = frozendict({'node_a1':1,'1':5,'node_z':4,'3':2})

## Parsing Test Cases

In [ ]:
#| export 
from typing import Dict, Any,Optional, Union, List
from pathlib import Path
from pprint import pprint
import yaml

In [ ]:
#| export

class Condition(BaseModel):
    key: str 
    value: Any
    func: Optional[str] = None
    kwargs: Dict[str,Any] = {}
    aggregation: Optional[str] = None
    
    @field_validator('key')
    @classmethod
    def validate_key(cls,v:str):
        try:
            parse_accessor(v)
        except Exception as e:
            raise ValueError(f"Invalid accessor: {v}, {e}") from e
        return v

    @field_validator('aggregation')
    @classmethod
    def validate_aggregation(cls,v:Optional[str]):
        if v is not None:
            if v not in ['min','max','sum','avg']:
                raise ValueError(f"Invalid aggregation: {v}, must be one of min,max,sum,avg")
        return v

class TestNode(BaseModel):
    name: str
    label: Optional[str] = None
    conditions: List[Condition]
    before: Optional[List[str]] = Field(default_factory=list)
    after: Optional[List[str]] = Field(default_factory=list)
    parallel: Optional[bool] = False

class TestCase(BaseModel):
    inputs: List[Any]
    test_nodes: List[TestNode]


def parse_test_case(raw_case):
    if isinstance(raw_case,Path):
        raw_case = raw_case.read_text()
    case_obj = TestCase.model_validate(yaml.safe_load(raw_case))
    for idx,node in enumerate(case_obj.test_nodes):
        if node.label is None:
            node.label = f"{idx}"
    return case_obj

In [ ]:
with pytest.raises(ValueError,match='Invalid accessor'):
    Condition(key='a.b.#c',value=1)


In [ ]:
t_case = parse_test_case(example_case)
t_case

TestCase(inputs=[{'content': 'hello world'}], test_nodes=[TestNode(name='node_a', label='node_a1', conditions=[Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_.*', label='node_z', conditions=[Condition(key='.', value='store', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='3', conditions=[Condition(key='b.c', value='store', func=None, kwargs={}, aggregation=None)], before=['node_z'], after=['node_a1'], parallel=False)])

In [ ]:
t_case = parse_test_case(example_case)
print(t_case)

inputs=[{'content': 'hello world'}] test_nodes=[TestNode(name='node_a', label='node_a1', conditions=[Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_.*', label='node_z', conditions=[Condition(key='.', value='store', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='3', conditions=[Condition(key='b.c', value='store', func=None, kwargs={}, aggregation=None)], before=['node_z'], after=['node_a1'], parallel=False)]


In [ ]:
#| export
# subset of stringdale.Trace that only contains the name and output
# since we only need them for evaluation
class SubTrace(BaseModel):
    model_config = ConfigDict(extra='allow')
    name: str
    output: Any


class TraceLog(BaseModel):
    steps: List[SubTrace]

In [ ]:
t_log = TraceLog(steps=example_trace_log)
t_log

TraceLog(steps=[SubTrace(name='Start', output='hello world'), SubTrace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}), SubTrace(name='node_c', output={'b': {'c': 'store is good'}}), SubTrace(name='node_a2', output={'b': {'d': 'store'}}), SubTrace(name='node_x', output='store'), SubTrace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}}), SubTrace(name='node_c', output={'b': {'c': 'store is good but not good enough'}}), SubTrace(name='node_y', output='stores')])

## Event stream warping

There are 3 steps to this algorithm:
* Compute distances between all Traces and all test nodes
* Solve a CSP with the partial order between test nodes to get potential assignments from trace nodes to traces
* Get the assignment with minimal distance

In [ ]:
#| export
from stringdale.core import maybe_await
from collections.abc import Iterable

In [ ]:
eval_funcs = {
    "regex": regex,
    "word_overlap": word_overlap,
}
default_func = 'word_overlap'

### Compute distances

In [ ]:
#| export

async def compute_condition_distance(trace:SubTrace,condition:Condition,eval_funcs,default_func):
    condition_func = eval_funcs.get(condition.func, eval_funcs[default_func])
    output_sub_value = access(trace.output,condition.key)

    def error_message(e):
        logger.error(f"Error computing distance for:\n"
                        f"trace {trace.name}\n"
                        f"condition function {condition_func}\n"
                        f"with key {condition.key}\n"
                        f"output value {repr(output_sub_value)}\n"
                        f"expected value {repr(condition.value)}\n"
                        f"with error: {e}")


    if condition.aggregation is None:
        try:
            logger.debug(
                f"Computing condition distance for key '{condition.key}' with value '{output_sub_value}'\n"
                f"condition function {condition_func}\n"
                f"args = [{output_sub_value}, {condition.value}]\n"
                f"kwargs = {condition.kwargs}\n"
            )
            condition_distance = await maybe_await(condition_func,args=[output_sub_value, condition.value],kwargs=condition.kwargs)
            logger.debug(f"Condition distance for key '{condition.key}' with value '{output_sub_value}' is {condition_distance}")
            agg_meta = None
        except Exception as e:
            error_message(e)
            return np.inf,output_sub_value,None
    else: # aggregation
        if not isinstance(output_sub_value,Iterable):
            error_message(f"Output sub value is not iterable: {output_sub_value}")
            return np.inf,output_sub_value
        logger.debug(f"Computing condition distance for key '{condition.key}' aggregation '{condition.aggregation}' with value {output_sub_value}")
        sub_condition_tasks = [maybe_await(condition_func,args=[v, condition.value],kwargs=condition.kwargs) for v in output_sub_value]
        distances = await asyncio.gather(*sub_condition_tasks)
        agg_meta = {
            "distances": distances,
            "values": output_sub_value,
        }

        if condition.aggregation == 'min':
            condition_distance = min(distances)
        elif condition.aggregation == 'max':
            condition_distance = max(distances)
        elif condition.aggregation == 'sum':
            condition_distance = sum(distances)
        elif condition.aggregation == 'avg':
            condition_distance = sum(distances) / len(distances)
        else:
            raise ValueError(f"Invalid aggregation: {condition.aggregation}")
    
    return condition_distance,output_sub_value,agg_meta


In [ ]:
trace = t_log.steps[1] 
cond = t_case.test_nodes[0].conditions[0]
trace,cond

(SubTrace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}),
 Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None))

In [ ]:
res,_,_ = await compute_condition_distance(trace,cond,eval_funcs,default_func)
assert res == 0.375

In [ ]:
listlike_trace = SubTrace(name="listlike_trace",output={'a':['1','2','3'],'b':[4,5,6]})

agg_cond = Condition(key='a',value='1',comparison='regex',aggregation='min')
regular_cond = Condition(key='a',value='1',comparison='regex')

res,_,_ = await compute_condition_distance(listlike_trace,agg_cond,eval_funcs,default_func)
assert res == 0

res,_,_ = await compute_condition_distance(listlike_trace,regular_cond,eval_funcs,default_func)
assert res == np.inf

In [ ]:
#| export
async def compute_node_distance(trace:SubTrace,node:TestNode,eval_funcs,default_func):

    logger.debug(f"Computing distance for trace {trace} and TestNode {node}")
    if not re.search(node.name, trace.name):
        return None,[]
    
    # check if all accessors are in the trace
    for condition in node.conditions:
        try: 
            access(trace.output,condition.key)
        except Exception as e:
            return np.inf, []

    debug_info = []

    distances,values,agg_metas = zip(*await await_all(
        [
            compute_condition_distance(trace,condition,eval_funcs,default_func)
            for condition in node.conditions
        ],
        error_prefix=[
            f"When computing distance for trace {trace.name} and node {node.name}"
            for condition in node.conditions
        ]
    ))

    logger.debug(f"Distances: {distances} values: {values}")
    distance = sum(distances)
    
    debug_info = []

    for condition,condition_distance,value,agg_meta in zip(node.conditions,distances,values,agg_metas):
        debug_info.append({
            "func": condition.func,
            "kwargs": condition.kwargs,
            "expected": condition.value,
            "actual": value,
            "key": condition.key,
            "distance": condition_distance,
            "aggregation": condition.aggregation,
            "agg_meta": agg_meta,
        })
        
    return distance,debug_info


In [ ]:
node = t_case.test_nodes[1]
node

TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False)

In [ ]:
trace = t_log.steps[-3]
trace

SubTrace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}})

In [ ]:
dist,debug_info = await compute_node_distance(trace,node,eval_funcs,default_func)
assert dist == 0,dist
debug_info

[{'func': 'regex',
  'kwargs': {},
  'expected': 'jimmy',
  'actual': 'jimmy',
  'key': 'd.e',
  'distance': 0,
  'aggregation': None,
  'agg_meta': None},
 {'func': 'chat',
  'kwargs': {'case_sensitive': False},
  'expected': 'is a good boy',
  'actual': 'is a good boy',
  'key': 'f.g',
  'distance': 0.0,
  'aggregation': None,
  'agg_meta': None}]

In [ ]:
#| export
async def compute_distances(
    trace_log:TraceLog,
    test_case:TestCase,
    comparisons:Dict[str,Callable],
    default_comparison:Callable,
    ):
    """
    Compute the distance matrix between the traces and the expected traces.

    Args:
        trace_log: TraceLog, the trace log
        test_case: TestCase, the test case
        comparisons: Dict[str,Callable], the comparisons to use for the distance matrix
        default_comparison: Callable, the default comparison to use for the distance matrix
    """
    nodes = test_case.test_nodes
    distances = dict()
    for node in nodes:
        distances[node.label] = dict()
    debug_info = defaultdict(dict)
    
    a_iter = list(it.product(enumerate(trace_log.steps), enumerate(nodes)))
    tasks = [
        compute_node_distance(trace,node,comparisons,default_comparison)
        for (i, trace), (j, node) in a_iter
    ]
    error_prefixes = [
        f"When computing distance for trace {trace.name}(#{i}) and node Test {node.name}(#{j})"
        for (i, trace), (j, node) in a_iter
    ]
    
    distance_list = await await_all(tasks,error_prefixes)

    for ((i, trace), (j, node)), (d,debug) in zip(a_iter, distance_list):
        if not d == None:
            if not d == np.inf:
                distances[node.label][i] = d
            debug_info[node.label][i]={
                'comparisons':debug,
                'distance':d,
                'node_idx':j,
                'trace_idx':i,
                'trace_name':trace.name,
                'node_name':node.name,
                'node_label':node.label,
            }

    return dict(distances),dict(debug_info)
    

In [ ]:
# with checkLogs():
dist,debug_info = await compute_distances(t_log,t_case,eval_funcs,default_func)
dist

{'node_a1': {1: 0.375},
 '1': {5: 0.0},
 'node_z': {4: 0.0, 7: 1.0},
 '3': {2: 0.6666666666666667, 6: 0.8333333333333334}}

In [ ]:
# debug_info

In [ ]:
expected_dist = {'node_a1': {1: 0.375},
 '3': {2: 0.6666666666666667, 6: 0.8333333333333334},
 'node_z': {4: 0.0, 7: 1.0},
 '1': {5: 0.0}}

assert dist == expected_dist

### CSM for possible assignments

In [ ]:
label_to_var = LabelToVar()
for idx,node in enumerate(t_case.test_nodes):
    label_to_var.add_label(node.label,idx)
label_to_var

<__main__.LabelToVar>

In [ ]:
#| export
from collections import defaultdict

In [ ]:
#| export

csp_logger = logging.getLogger(f'{__name__}.csp')

def get_possible_assignments(dist,test_case:TestCase,label_to_var:LabelToVar):
    """
    Gets possible mappings between expected traces and actual traces.
    By building a constraint satisfaction problem and solving it.
    """
    p = Problem()
    csp_logger.debug(
        f"Adding variables for {test_case.test_nodes}\n"
        f"dist: {dist}"
        f"label_to_var: {label_to_var}"
        )


    # TODO if a<b and c is after b and is parallel, we need to deduce that c is bounded by the bound of a

    # we track upper and lower bounds, so that we can deduce the bounds
    #  of parallel nodes by the bounds of the node before them
    lower_bounds = defaultdict(set)
    upper_bounds = defaultdict(set)
    
    for col_idx,node in enumerate(test_case.test_nodes):
        viable_trace_row_nums = list(dist[node.label].keys())
        if not viable_trace_row_nums:
            csp_logger.warning(f"No viable trace row nums for expected trace {node.label}")
            return None, p
        var_name = label_to_var.get_col(node.label)
        p.addVariable(var_name,viable_trace_row_nums)
        csp_logger.debug(f"Adding variable {var_name} with domain {viable_trace_row_nums}")

        explicit_constraints = len(node.before) + len(node.after) > 0
        if explicit_constraints:
            for before_label in node.before:
                before_var_name = label_to_var.get_col(before_label)
                csp_logger.debug(f"Adding constraint {before_var_name} < {var_name}")
                p.addConstraint(f"{var_name} < {before_var_name}")
                lower_bounds[var_name].add(before_var_name)

            for after_label in node.after:
                after_var_name = label_to_var.get_col(after_label)
                csp_logger.debug(f"Adding constraint {var_name} < {after_var_name}")
                p.addConstraint(f"{after_var_name} < {var_name}")
                upper_bounds[var_name].add(after_var_name)
        else:

            if col_idx == 0:
                continue
            
            before_var_name = label_to_var.get_col(test_case.test_nodes[col_idx-1].label)
            if not node.parallel:
                csp_logger.debug(f"Adding constraint {before_var_name} < {var_name}")
                p.addConstraint(f"{before_var_name} < {var_name}")
                lower_bounds[var_name].add(before_var_name)

            if node.parallel:
                # derive the bounds from the bounds of the node before
                for lower_bound in lower_bounds[before_var_name]:
                    csp_logger.debug(f"Adding constraint {lower_bound} < {var_name} (derived from {before_var_name} through parallel node {node.label})")
                    p.addConstraint(f"{lower_bound} < {var_name}")
                    lower_bounds[var_name].add(lower_bound)
                for upper_bound in upper_bounds[before_var_name]:
                    csp_logger.debug(f"Adding constraint {var_name} < {upper_bound} (derived from {before_var_name} through parallel node {node.label})")
                    p.addConstraint(f"{var_name} < {upper_bound}")
                    upper_bounds[var_name].add(upper_bound)
        
    # these solutions use colnames    
    solutions = p.getSolutions()
    # invert the colnames back to labels
    labeled_solutions = set(frozendict({label_to_var.get_label(k):v for k,v in sol.items()}) for sol in solutions)
    return labeled_solutions, p

In [ ]:
from deepdiff import DeepDiff
from copy import deepcopy

In [ ]:
dist

{'node_a1': {1: 0.375},
 '1': {5: 0.0},
 'node_z': {4: 0.0, 7: 1.0},
 '3': {2: 0.6666666666666667, 6: 0.8333333333333334}}

In [ ]:
dist2 = deepcopy(dist)
dist2['node_a1'] = dict()
dist2

{'node_a1': {},
 '1': {5: 0.0},
 'node_z': {4: 0.0, 7: 1.0},
 '3': {2: 0.6666666666666667, 6: 0.8333333333333334}}

In [ ]:
label_to_var 

<__main__.LabelToVar>

In [ ]:
with checkLogs():
    labeled_solutions,p = get_possible_assignments(dist2,t_case,label_to_var)
assert labeled_solutions is None

__main__.csp - DEBUG - Adding variables for [TestNode(name='node_a', label='node_a1', conditions=[Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_.*', label='node_z', conditions=[Condition(key='.', value='store', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='3', conditions=[Condition(key='b.c', value='store', func=None, kwargs={}, aggregation=None)], before=['node_z'], after=['node_a1'], parallel=False)]
dist: {'node_a1': {}, '1': {5: 0.0}, 'node_z': {4: 0.0, 7: 1.0}, '3': {2: 0.6666666666666667, 6: 0.8333333333333334}}label_to_var: <__main__.La

In [ ]:
labeled_solutions

In [ ]:
with checkLogs():
    labeled_solutions,p = get_possible_assignments(dist,t_case,label_to_var)

diff = DeepDiff(labeled_solutions,possible_mappings)
assert diff == {}, diff
labeled_solutions

__main__.csp - DEBUG - Adding variables for [TestNode(name='node_a', label='node_a1', conditions=[Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_.*', label='node_z', conditions=[Condition(key='.', value='store', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='3', conditions=[Condition(key='b.c', value='store', func=None, kwargs={}, aggregation=None)], before=['node_z'], after=['node_a1'], parallel=False)]
dist: {'node_a1': {1: 0.375}, '1': {5: 0.0}, 'node_z': {4: 0.0, 7: 1.0}, '3': {2: 0.6666666666666667, 6: 0.8333333333333334}}label_to_var: <__m

{frozendict.frozendict({'node_a1': 1, 'node_z': 4, '3': 2, '1': 5}),
 frozendict.frozendict({'node_a1': 1, 'node_z': 7, '3': 2, '1': 5}),
 frozendict.frozendict({'node_a1': 1, 'node_z': 7, '3': 6, '1': 5})}

In [ ]:
#| export
def get_best_assignment(dist_matrix,possible_mappings,label_to_var):
    """
    dist_matrix: np.ndarray
    possible_mappings: list of tuples
    label_to_var: dict
    """
    
    score_per_solution = {}
    for sol in possible_mappings:
        sum_dist = 0
        for expected_label,trace_idx in sol.items():
            sum_dist += dist_matrix[expected_label][trace_idx]
        score_per_solution[sol] = sum_dist

    best_solution =  min(score_per_solution,key=score_per_solution.get)
    best_solution_score = score_per_solution[best_solution]
    return best_solution,best_solution_score

In [ ]:
best_mapping,best_score = get_best_assignment(dist,possible_mappings,label_to_var)
assert best_mapping == frozendict({'node_a1': 1,'3': 2,'node_z': 4,'1': 5})

In [ ]:
#| export
async def event_stream_warp(trace_log,test_case,eval_funcs,default_func):
    """
    Perform event stream warping on a trace log and a test case.

    Args:
        trace_log: TraceLog, the trace log to warp
        test_case: TestCase, the test case to warp to
        eval_funcs: Dict[str,Callable], the dictionary of functions to use for evaluation
        default_func: Callable, the default function to use

    Returns:
        best_mapping: Dict[str,int], the best mapping from test_node labels to trace indexes
        best_score: float, the score of the best mapping
        debug_info: List[Dict[str,Any]], debug information for comparisons between all test nodes and all traces
        dist: Dict[str,Dict[int,float]], the distance matrix
        problem: Problem, the constraint satisfaction problem
    """
    label_to_var = LabelToVar()
    for idx,node in enumerate(test_case.test_nodes):
        label_to_var.add_label(node.label,idx)

    dist,debug_info = await compute_distances(trace_log,test_case,eval_funcs,default_func)
    possible_mappings,problem = get_possible_assignments(dist,test_case,label_to_var)
    if not possible_mappings:
        return None, np.inf, debug_info, dist, problem
    best_mapping,best_score = get_best_assignment(dist,possible_mappings,label_to_var)
    return best_mapping, best_score, debug_info, dist, problem



## End to end test

In [ ]:
# t_case,t_log

In [ ]:

default_comparison = 'word_overlap'
comparisons = {
    "regex": regex,
    "word_overlap": word_overlap,
}

with checkLogs(name='__main__.csp'):
    aligned_traces,score,debug_info,dist,problem = await event_stream_warp(t_log,t_case,eval_funcs,default_func)
aligned_traces,score

__main__.csp - DEBUG - Adding variables for [TestNode(name='node_a', label='node_a1', conditions=[Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_.*', label='node_z', conditions=[Condition(key='.', value='store', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='3', conditions=[Condition(key='b.c', value='store', func=None, kwargs={}, aggregation=None)], before=['node_z'], after=['node_a1'], parallel=False)]
dist: {'node_a1': {1: 0.375}, '1': {5: 0.0}, 'node_z': {4: 0.0, 7: 1.0}, '3': {2: 0.6666666666666667, 6: 0.8333333333333334}}label_to_var: <__m

(frozendict.frozendict({'node_a1': 1, 'node_z': 4, '3': 2, '1': 5}),
 1.0416666666666667)

In [ ]:
debug_info.keys()

dict_keys(['node_a1', 'node_z', '3', '1'])

In [ ]:
assert aligned_traces == frozendict({'node_a1': 1,'3': 2,'node_z': 4,'1': 5})

## Parallel case

In [ ]:
t_log

TraceLog(steps=[SubTrace(name='Start', output='hello world'), SubTrace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}), SubTrace(name='node_c', output={'b': {'c': 'store is good'}}), SubTrace(name='node_a2', output={'b': {'d': 'store'}}), SubTrace(name='node_x', output='store'), SubTrace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}}), SubTrace(name='node_c', output={'b': {'c': 'store is good but not good enough'}}), SubTrace(name='node_y', output='stores')])

In [ ]:
t_case

TestCase(inputs=[{'content': 'hello world'}], test_nodes=[TestNode(name='node_a', label='node_a1', conditions=[Condition(key='b.c', value='jimmy went\nto the store\n', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='d.e', value='jimmy', func='regex', kwargs={}, aggregation=None), Condition(key='f.g', value='is a good boy', func='chat', kwargs={'case_sensitive': False}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_.*', label='node_z', conditions=[Condition(key='.', value='store', func=None, kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='3', conditions=[Condition(key='b.c', value='store', func=None, kwargs={}, aggregation=None)], before=['node_z'], after=['node_a1'], parallel=False)])

In [ ]:
import rich

In [ ]:
without_parallel_expected = parse_test_case("""
inputs:
  - anything
test_nodes:
  - name: node_a
    conditions:
      - key: .
        func: regex
        value: a
  - name: node_b
    conditions:
      - key: .
        func: regex
        value: b
  - name: node_c
    conditions:
      - key: .
        func: regex
        value: c
""")
rich.print(without_parallel_expected)

TestCase(
    inputs=['anything'],
    test_nodes=[
        TestNode(
            name='node_a',
            label='0',
            conditions=[Condition(key='.', value='a', func='regex', kwargs={}, aggregation=None)],
            before=[],
            after=[],
            parallel=False
        ),
        TestNode(
            name='node_b',
            label='1',
            conditions=[Condition(key='.', value='b', func='regex', kwargs={}, aggregation=None)],
            before=[],
            after=[],
            parallel=False
        ),
        TestNode(
            name='node_c',
            label='2',
            conditions=[Condition(key='.', value='c', func='regex', kwargs={}, aggregation=None)],
            before=[],
            after=[],
            parallel=False
        )
    ]
)

In [ ]:
right_order_log = TraceLog(
    steps=[
        {"name":"node_a","output":"a"},
        {"name":"node_b","output":"b"},
        {"name":"node_c","output":"c"},
    ]
)
right_order_log 

TraceLog(steps=[SubTrace(name='node_a', output='a'), SubTrace(name='node_b', output='b'), SubTrace(name='node_c', output='c')])

In [ ]:
with checkLogs(name='__main__.csp'):
    aligned_traces,score,debug_info,dist,problem = await event_stream_warp(right_order_log,without_parallel_expected,eval_funcs,default_func)
aligned_traces,score
assert aligned_traces == frozendict({'0': 0,'1': 1,'2': 2})
assert score == 0

__main__.csp - DEBUG - Adding variables for [TestNode(name='node_a', label='0', conditions=[Condition(key='.', value='a', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='.', value='b', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_c', label='2', conditions=[Condition(key='.', value='c', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False)]
dist: {'0': {0: 0}, '1': {1: 0}, '2': {2: 0}}label_to_var: <__main__.LabelToVar object>
__main__.csp - DEBUG - Adding variable a with domain [0]
__main__.csp - DEBUG - Adding variable b with domain [1]
__main__.csp - DEBUG - Adding constraint a < b
__main__.csp - DEBUG - Adding variable c with domain [2]
__main__.csp - DEBUG - Adding constraint b < c


In [ ]:
wrong_order_log = TraceLog(
    steps=[
        {"name":"node_c","output":"c"},
        {"name":"node_a","output":"a"},
        {"name":"node_b","output":"b"},
    ]
)
wrong_order_log 

TraceLog(steps=[SubTrace(name='node_c', output='c'), SubTrace(name='node_a', output='a'), SubTrace(name='node_b', output='b')])

In [ ]:
with checkLogs(name='__main__.csp'):
    aligned_traces,score,debug_info,dist,problem = await event_stream_warp(wrong_order_log,without_parallel_expected,eval_funcs,default_func)
aligned_traces,score
assert aligned_traces == None, aligned_traces

__main__.csp - DEBUG - Adding variables for [TestNode(name='node_a', label='0', conditions=[Condition(key='.', value='a', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='.', value='b', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_c', label='2', conditions=[Condition(key='.', value='c', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False)]
dist: {'0': {1: 0}, '1': {2: 0}, '2': {0: 0}}label_to_var: <__main__.LabelToVar object>
__main__.csp - DEBUG - Adding variable a with domain [1]
__main__.csp - DEBUG - Adding variable b with domain [2]
__main__.csp - DEBUG - Adding constraint a < b
__main__.csp - DEBUG - Adding variable c with domain [0]
__main__.csp - DEBUG - Adding constraint b < c


In [ ]:
parallel_expected = parse_test_case("""
inputs:
  - anything
test_nodes:
  - name: node_a
    conditions:
      - key: .
        func: regex
        value: a
  - name: node_b
    conditions:
      - key: .
        func: regex
        value: b
    parallel: true
  - name: node_c
    conditions:
      - key: .
        func: regex
        value: c
    parallel: true
""")
rich.print(parallel_expected)

TestCase(
    inputs=['anything'],
    test_nodes=[
        TestNode(
            name='node_a',
            label='0',
            conditions=[Condition(key='.', value='a', func='regex', kwargs={}, aggregation=None)],
            before=[],
            after=[],
            parallel=False
        ),
        TestNode(
            name='node_b',
            label='1',
            conditions=[Condition(key='.', value='b', func='regex', kwargs={}, aggregation=None)],
            before=[],
            after=[],
            parallel=True
        ),
        TestNode(
            name='node_c',
            label='2',
            conditions=[Condition(key='.', value='c', func='regex', kwargs={}, aggregation=None)],
            before=[],
            after=[],
            parallel=True
        )
    ]
)

In [ ]:
with checkLogs(name='__main__.csp'):
    aligned_traces,score,debug_info,dist,problem = await event_stream_warp(wrong_order_log,parallel_expected,eval_funcs,default_func)
assert aligned_traces == frozendict({'0': 1,'1': 2,'2': 0}), aligned_traces
aligned_traces,score


__main__.csp - DEBUG - Adding variables for [TestNode(name='node_a', label='0', conditions=[Condition(key='.', value='a', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=False), TestNode(name='node_b', label='1', conditions=[Condition(key='.', value='b', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=True), TestNode(name='node_c', label='2', conditions=[Condition(key='.', value='c', func='regex', kwargs={}, aggregation=None)], before=[], after=[], parallel=True)]
dist: {'0': {1: 0}, '1': {2: 0}, '2': {0: 0}}label_to_var: <__main__.LabelToVar object>
__main__.csp - DEBUG - Adding variable a with domain [1]
__main__.csp - DEBUG - Adding variable b with domain [2]
__main__.csp - DEBUG - Adding variable c with domain [0]


(frozendict.frozendict({'0': 1, '1': 2, '2': 0}), 0)

## export

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()